In [23]:
from typing import TypedDict, List
import random as r
from langgraph.graph import StateGraph, START, END

In [24]:
class AgentState(TypedDict):
    player_name:str
    guesses:List[int]
    attempts:int
    lower_bound:int
    upper_bound:int
    target:int

In [25]:
def setup(state:AgentState) -> AgentState:
    state['attempts']=-1
    state['guesses']=[]
    state['lower_bound']=1
    state['upper_bound']=20
    print(f"Your name is {state['player_name']} and please start the game!")
    return state

def guess(state:AgentState) -> AgentState:
    state['guesses']=state['guesses'].append(r.randint(state['lower_bound'], state['upper_bound']))
    state['attempts']+=1
    return state

def hint(state:AgentState) -> AgentState:
    if (state['attempts']<7) :
        if state['guesses'][state['attempts']]>state['target']:
            print(f"higher, {state['attempts']} attempts so far")
            state['upper_bound']-=1
            state['attempts']+=1
            return "continue"
        elif state['guesses'][state['attempts']]<state['target']:
            print(f"lower, {state['attempts']} attempts so far")
            state['lower_bound']+=1
            state['attempts']+=1
            return "continue"
        else:
            print("target found")
            return "end"
        
    else:
        print("target not found in 7 iterations")
        return "end"
        



In [26]:
graph=StateGraph(AgentState)
graph.add_node("setup", setup)
graph.add_node("guessing", guess)
graph.add_node("hint_code", hint)

graph.add_edge(START, "setup")
graph.add_edge("setup", "guessing")
graph.add_edge("guessing", "hint_code")

graph.add_conditional_edges("hint_code", hint, {
    "end":END, "continue":"guessing"
})

app=graph.compile()

In [27]:
app.invoke({"player_name":"Inder", "attempts":0, "lower_bound":1, "upper_bound":20, "guesses":[], "target":6})

Your name is Inder and please start the game!


TypeError: 'NoneType' object is not subscriptable